In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from scipy import pi
from scipy.io import loadmat
from scipy.fft import fft, ifft, fftfreq

In [ ]:
import pandas as pd

In [ ]:
from ssvepcca.definitions import (
    NUM_BLOCKS,
    NUM_TARGETS,
    NUM_SAMPLES,
    NUM_ELECTRODES,
    SAMPLE_FREQ,
    SAMPLE_T,
    TARGET_FREQUENCY,
    TARGET_PHASE,
    ELECTRODE_INDEX
)

In [ ]:
from ssvepcca.utils import (_get_time_column,
                   # transform_mat_to_pandas,
                  get_harmonic_columns)

# Exploring Sample

## Parsing input dataset for S1 (subject 1)

In [ ]:
s1 = loadmat("dataset_chines/S1.mat")

In [ ]:
s1.keys()

In [ ]:
s1['__header__']

In [ ]:
s1['__version__']

In [ ]:
s1['__globals__']

In [ ]:
m1 = s1['data']

In [ ]:
m1.shape

In [ ]:
m1[:, :, 0, 0]

In [ ]:
df1 = transform_mat_to_pandas(m1)

In [ ]:
df1

In [ ]:
df = df1.set_index(["block", "target", "time_ms"])

In [ ]:
df.loc[0,0].loc[0:1000]["electrode_0"].plot()

In [ ]:
df.loc[0,0]["electrode_20"].plot()

In [ ]:
df.loc[0,6]["electrode_20"].plot()

## Playing with FFT, just to see what happens

In [ ]:
N = NUM_SAMPLES # Number of sample points
T = SAMPLE_T # sample spacing 
y = df.loc[0,6]["electrode_50"].values

yf = fft(y)
xf = fftfreq(N, T)[:N//8]
import matplotlib.pyplot as plt
plt.plot(xf, 2.0/N * np.abs(yf[:N//8]))
plt.grid()
plt.show()

In [ ]:
from scipy.signal import blackman
import matplotlib.pyplot as plt

xf = fftfreq(N, T)[:N//2]
yf = fft(y)
w = blackman(N)
ywf = fft(y*w)

plt.semilogy(xf[1:N//8], 2.0/N * np.abs(yf[1:N//8]), '-b')
plt.semilogy(xf[1:N//8], 2.0/N * np.abs(ywf[1:N//8]), '-r')
plt.legend(['FFT', 'FFT w. window'])
plt.grid()
plt.show()

## Exploring Phase Dataset

In [ ]:
freqphase = loadmat("dataset_chines/Freq_Phase.mat")

In [ ]:
freqphase

In [ ]:
freqphase["phases"].shape

In [ ]:
freqphase["freqs"].reshape(-1, 8)

In [ ]:
freqphase["phases"].round(4).reshape(-1, 8)

# POC with CCA (original algorithm)

**Plan:**

- get sin and cos components (6 components/columns)
- for each observation, we are going to estimate the correlation with all possible frequencies.

In [ ]:
np.sin(2*pi)

In [ ]:
freqs = freqphase["freqs"].reshape(-1)
phases =  freqphase["phases"].reshape(-1)

In [ ]:
BLOCK = 0
TARGET = 0

In [ ]:
obs_dataframe = df.loc[BLOCK, TARGET]

In [ ]:
obs_dataframe

In [ ]:
obs_dataframe['time_index'] * 1 / 250

In [ ]:
m1.shape

In [ ]:
trial = m1[:, :, TARGET, BLOCK].T

In [ ]:
trial.shape

In [ ]:
get_harmonic_columns(2).shape

In [ ]:
harmonics_8 = get_harmonic_columns(8)

In [ ]:
harmonics_8.shape

In [ ]:
from sklearn.cross_decomposition import CCA


In [ ]:
from definitions import ELECTRODE_INDEX
from parameters import electrode_list_fbcca

In [ ]:
electrode_index = [ELECTRODE_INDEX[electrode_name] for electrode_name in electrode_list_fbcca]

In [ ]:
from learners import CCASingleComponent, CCACorrelation

In [ ]:
trial[:, electrode_index].shape

In [ ]:
m = CCASingleComponent()
m.predict_proba(trial[:, electrode_index])

In [ ]:
m = CCASingleComponent()
m.predict(trial[:, electrode_index])

In [ ]:
model = CCACorrelation(n_components=2)

model.fit(trial[:, electrode_index], get_harmonic_columns(8))

In [ ]:
trial

In [ ]:
model.fit_correlation(trial[:, electrode_index], get_harmonic_columns(8))

In [ ]:
get_harmonic_columns?

In [ ]:
model2 = CCACorrelation(n_components=2)
model2.fit_correlation(trial[:, electrode_index], get_harmonic_columns(8))

In [ ]:
import statsmodels.multivariate.cancorr

In [ ]:
cca_stats = statsmodels.multivariate.cancorr.CanCorr(
    endog = get_harmonic_columns(8),
    exog = trial[:, electrode_index],
)
cca_stats.cancorr

## Design experiments

We need to benchmark different algorithms.
For this, we need to do a workflow of:

- For each subject (35)
    - For each key (40)
        - If the model need training
            - For each BLOCK (6)
                - Train a model on the other 5 BLOCKS (leave-one-out scheme)
                - Predict the label of this trial
                - Store prediction in a response object
        - If the model doesn't need training
            - For each BLOCK (6)
                - Predict label of this BLOCK
                - Store prediction in a response object

*We need to filter the electrodes somewhere. It can be learned for each subject. If learned, it should be done within the leave-one-out scheme.*

Output: matrix with dimension (40, 6) for each subject

Create a function to calculate:

- accuracy
- confusion matrix

In [ ]:
m1[0, 0, 0, :]

In [ ]:
np.identity(6, dtype=bool)[0]

In [ ]:
masked = (np.identity(6)[0] - 1) * -1

In [ ]:
masked

In [ ]:
m1[:, :, :, ~np.identity(6, dtype=bool)[5]].shape

In [ ]:
m1_key = m1[:, :, 0, :]

In [ ]:
m1_key.shape

In [ ]:
np.empty([5])

In [ ]:
m1_key.shape == (64, 1500, 6)

**Learner**

The learner should comprise everything needed to estimate a prediction. It should be a full pipeline.
If, for example, it is necessary to apply a filtering technique, such as filterbank, it should be learned and applied by the same object. It is an object that can learn everything necessary.

It can be either an object or a curried function. Good moment to learn how to use SKLEARN PIPELINES and custom transformers. Custom transformers are a very cool feature to use.

If need to learn with labels, it should be trained with other observations with same target.

In [ ]:
from pipelines import *

In [ ]:
test_fit_predict

In [ ]:
M1 = m1.T

In [ ]:
M1.shape

In [ ]:
M1[0, 0, :, :]

In [ ]:
m = CCASingleComponent()

In [ ]:
r = test_fit_predict(m, M1)

In [ ]:
r.shape

In [ ]:
r[:, 0]

In [ ]:
r.T.shape

In [ ]:
r.T[0, 0]

In [ ]:
r.T

In [ ]:
count = 0
for l in range(40):
    for c in range(6):
        if r.T[l, c] == l:
            count += 1

count, count/(40*6)

In [ ]:
np.array(TARGET_FREQUENCY).reshape(5, -1)

In [ ]:
np.array(range(40)).reshape(5,-1)

In [ ]:
np.array(TARGET_PHASE).round(3).reshape(5, -1)